# Parenteral Dataset Quality Analysis

This notebook analyzes the quality and characteristics of the parenteral medication image dataset.

**Objectives:**
1. Generate statistical analysis of image properties
2. Identify missing or inconsistent data
3. Create visualizations of dataset distribution
4. Document data quality findings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style for visualizations
plt.style.use('seaborn')
sns.set_palette('deep')
plt.rcParams['figure.figsize'] = [12, 6]

## 1. Data Loading and Initial Inspection

In [ ]:
# Load the dataset
df = pd.read_csv('parenteral_metadata.csv')

print(f"Dataset shape: {df.shape}")
print("\nColumn info:")
df.info()

## 2. Image Properties Analysis

In [ ]:
# Image dimensions analysis
def analyze_image_dimensions(df):
    dim_stats = pd.DataFrame({
        'Width': df['image_width'].describe(),
        'Height': df['image_height'].describe()
    })
    
    # Calculate aspect ratios
    df['aspect_ratio'] = df['image_width'] / df['image_height']
    
    # Plot dimension distributions
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    sns.histplot(data=df, x='image_width', bins=30, ax=ax1)
    ax1.set_title('Width Distribution')
    
    sns.histplot(data=df, x='image_height', bins=30, ax=ax2)
    ax2.set_title('Height Distribution')
    
    sns.histplot(data=df, x='aspect_ratio', bins=30, ax=ax3)
    ax3.set_title('Aspect Ratio Distribution')
    
    plt.tight_layout()
    
    return dim_stats

dim_stats = analyze_image_dimensions(df)
print("Dimension Statistics:")
print(dim_stats)

In [ ]:
# Image format and file size analysis
def analyze_image_formats(df):
    # Format distribution
    format_dist = df['image_format'].value_counts()
    
    # File size statistics by format
    size_by_format = df.groupby('image_format')['file_size_kb'].describe()
    
    # Visualize format distribution
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    format_dist.plot(kind='bar', ax=ax1)
    ax1.set_title('Image Format Distribution')
    ax1.set_ylabel('Count')
    
    sns.boxplot(data=df, x='image_format', y='file_size_kb', ax=ax2)
    ax2.set_title('File Size Distribution by Format')
    ax2.set_ylabel('File Size (KB)')
    
    plt.tight_layout()
    
    return format_dist, size_by_format

format_dist, size_by_format = analyze_image_formats(df)
print("Format Distribution:")
print(format_dist)
print("\nFile Size Statistics by Format:")
print(size_by_format)

## 3. Missing and Inconsistent Data Analysis

In [ ]:
def analyze_data_quality(df):
    # Missing values analysis
    missing_vals = df.isnull().sum()
    missing_percentages = (missing_vals / len(df) * 100).round(2)
    
    # Identify duplicate files
    duplicates = df[df.duplicated(subset=['file_name'], keep=False)]
    
    # Check for inconsistent dates
    df['date_added'] = pd.to_datetime(df['date_added'])
    df['last_modified'] = pd.to_datetime(df['last_modified'])
    date_issues = df[df['last_modified'] > df['date_added']]
    
    # Check for unusual dimensions or file sizes
    size_outliers = df[df['file_size_kb'] > df['file_size_kb'].quantile(0.99)]
    dimension_outliers = df[
        (df['image_width'] > df['image_width'].quantile(0.99)) |
        (df['image_height'] > df['image_height'].quantile(0.99))
    ]
    
    return {
        'missing_vals': pd.DataFrame({'Count': missing_vals, 'Percentage': missing_percentages}),
        'duplicate_count': len(duplicates),
        'date_issues_count': len(date_issues),
        'size_outliers': len(size_outliers),
        'dimension_outliers': len(dimension_outliers)
    }

quality_metrics = analyze_data_quality(df)
print("Missing Values Analysis:")
print(quality_metrics['missing_vals'])
print(f"\nNumber of duplicate files: {quality_metrics['duplicate_count']}")
print(f"Number of date inconsistencies: {quality_metrics['date_issues_count']}")
print(f"Number of file size outliers: {quality_metrics['size_outliers']}")
print(f"Number of dimension outliers: {quality_metrics['dimension_outliers']}")

## 4. Dataset Distribution Analysis

In [ ]:
def analyze_dataset_distribution(df):
    # Category distribution
    category_dist = df['category'].value_counts()
    
    # Concentration distribution
    concentration_dist = df['concentration'].value_counts()
    
    # Visualize distributions
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # Plot top 20 categories
    category_dist.head(20).plot(kind='barh', ax=ax1)
    ax1.set_title('Top 20 Categories Distribution')
    ax1.set_xlabel('Count')
    
    # Plot top 20 concentrations
    concentration_dist.head(20).plot(kind='barh', ax=ax2)
    ax2.set_title('Top 20 Concentrations Distribution')
    ax2.set_xlabel('Count')
    
    plt.tight_layout()
    
    return category_dist, concentration_dist

category_dist, concentration_dist = analyze_dataset_distribution(df)
print("Category Distribution Summary:")
print(f"Total number of categories: {len(category_dist)}")
print("\nTop 10 categories:")
print(category_dist.head(10))

## 5. Data Quality Summary Report

In [ ]:
def generate_quality_report(df, quality_metrics):
    report = f"""
    # Dataset Quality Analysis Report
    
    ## Dataset Overview
    - Total number of images: {len(df)}
    - Number of categories: {len(df['category'].unique())}
    - Date range: {df['date_added'].min()} to {df['date_added'].max()}
    
    ## Image Properties
    - Image formats: {', '.join(df['image_format'].unique())}
    - Average dimensions: {df['image_width'].mean():.0f}x{df['image_height'].mean():.0f} pixels
    - Average file size: {df['file_size_kb'].mean():.2f} KB
    
    ## Data Quality Issues
    - Missing values: {quality_metrics['missing_vals']['Count'].sum()} total across all columns
    - Duplicate files: {quality_metrics['duplicate_count']}
    - Date inconsistencies: {quality_metrics['date_issues_count']}
    - Size outliers: {quality_metrics['size_outliers']}
    - Dimension outliers: {quality_metrics['dimension_outliers']}
    
    ## Recommendations
    1. {'Address missing values' if quality_metrics['missing_vals']['Count'].sum() > 0 else 'No major missing value issues'}
    2. {'Review and resolve duplicate files' if quality_metrics['duplicate_count'] > 0 else 'No duplicate files found'}
    3. {'Investigate date inconsistencies' if quality_metrics['date_issues_count'] > 0 else 'No date inconsistencies found'}
    4. {'Review outlier images for quality issues' if quality_metrics['size_outliers'] > 0 else 'No significant outliers found'}
    """
    
    return report

quality_report = generate_quality_report(df, quality_metrics)
print(quality_report)